In [74]:
import threading
import time
import re
import json
from collections import OrderedDict

In [75]:
class TweetCache:
    def __init__(self, max_size=30):
        self.cache = OrderedDict()  
        self.id_list = []  
        self.max_size = max_size  
        self.refresh_time = 30 * 60
        self.last_refresh_time = time.time()

        self.refresh_thread = threading.Thread(target=self._auto_refresh_cache)
        self.refresh_thread.daemon = True
        self.refresh_thread.start()

    def _refresh_cache(self):
        print("Refreshing cache...")
        current_time = time.time()
        expired_items = []
        for query_id, data in self.cache.items():
            if current_time - data["timestamp"] >= self.refresh_time:
                expired_items.append(query_id)
        for query_id in expired_items:
            del self.cache[query_id]
            self.id_list.remove(query_id)  
        self.last_refresh_time = current_time

    def _normalize_query_input(self, query):
        normalized_values = {key: self._clean_text(str(value).lower()) for key, value in query.items()}
        return normalized_values

    def _clean_text(self, text):
        cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        return cleaned_text.strip()

    def _get_ui_input_concatenated(self, query):
        return ''.join(self._normalize_query_input(query).values())

    def add_to_cache(self, query, data):
        ui_input_concatenated = self._get_ui_input_concatenated(query)
        if ui_input_concatenated in self.cache:
            del self.cache[ui_input_concatenated]  
            self.id_list.remove(ui_input_concatenated)
        elif len(self.cache) >= self.max_size:  
            oldest_key = next(iter(self.cache))
            del self.cache[oldest_key]
            self.id_list.remove(oldest_key)
        self.cache[ui_input_concatenated] = {
            'ID': ui_input_concatenated,
            'Query': query,
            'data': data,
            'timestamp': time.time()  
        }
        self.id_list.append(ui_input_concatenated)  
        
    def get_from_cache(self, query):
        ui_input_concatenated = self._get_ui_input_concatenated(query)
        if ui_input_concatenated in self.cache:
            self.cache.move_to_end(ui_input_concatenated)  
            return self.cache[ui_input_concatenated]
        return None

    def display_cache(self):
        print("Current Cache Contents:")
        print("ID List:", self.id_list)
        print("--------------------")
        for query_id, data in self.cache.items():
            print("Query ID:", query_id)
            print("Query:", data["Query"])
            print("Data:", data["data"])
            print("Timestamp:", time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(data["timestamp"])))
            print("--------------------")

    def export_to_json(self, filename):
        cache_data = {"cache": self.cache, "id_list": self.id_list}
        with open(filename, 'w') as f:
            json.dump(cache_data, f)

    def _auto_refresh_cache(self):
        while True:
            current_time = time.time()
            if current_time - self.last_refresh_time >= self.refresh_time:
                self._refresh_cache()
                print("Cache refreshed at:", time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(current_time)))
            time.sleep(self.refresh_time / 2)

In [81]:
tweet_cache = TweetCache(max_size=30)

# Adding values to cache
for i in range(32):
    tweet_cache.add_to_cache({"username": f"user{i}", "age": str(i)}, {"tweet": f"example tweet {i}"})

In [82]:
# Displaying cache contents
tweet_cache.display_cache()

Current Cache Contents:
ID List: ['user22', 'user33', 'user44', 'user55', 'user66', 'user77', 'user88', 'user99', 'user1010', 'user1111', 'user1212', 'user1313', 'user1414', 'user1515', 'user1616', 'user1717', 'user1818', 'user1919', 'user2020', 'user2121', 'user2222', 'user2323', 'user2424', 'user2525', 'user2626', 'user2727', 'user2828', 'user2929', 'user3030', 'user3131']
--------------------
Query ID: user22
Query: {'username': 'user2', 'age': '2'}
Data: {'tweet': 'example tweet 2'}
Timestamp: 2024-04-22 11:39:03
--------------------
Query ID: user33
Query: {'username': 'user3', 'age': '3'}
Data: {'tweet': 'example tweet 3'}
Timestamp: 2024-04-22 11:39:03
--------------------
Query ID: user44
Query: {'username': 'user4', 'age': '4'}
Data: {'tweet': 'example tweet 4'}
Timestamp: 2024-04-22 11:39:03
--------------------
Query ID: user55
Query: {'username': 'user5', 'age': '5'}
Data: {'tweet': 'example tweet 5'}
Timestamp: 2024-04-22 11:39:03
--------------------
Query ID: user66
Quer

In [78]:

# Retrieving values from cache
print(tweet_cache.get_from_cache({"username": "user10", "age": "10"}))


{'ID': 'user1010', 'Query': {'username': 'user10', 'age': '10'}, 'data': {'tweet': 'example tweet 10'}, 'timestamp': 1713800132.802891}


In [67]:
# Example usage:
cache = TweetCache()

# Add data to cache
query = {'username': 'RED,', 'age': '20'}
data = [{'tweetid': 123, 'username': 'user1'},
        {'tweetid': 456, 'username': 'user2'}]

In [68]:
# Add data to cache
query1 = {'username': 'BLUE,', 'age': '21'}
data1 = [{'tweetid': 456, 'username': 'user3'},
        {'tweetid': 789, 'username': 'user4'}]

In [69]:
cache.add_to_cache(query, data)

In [70]:
cache.add_to_cache(query1, data1)

In [71]:
cache.display_cache()

Current Cache Contents:
ID List: ['red20', 'blue21']
--------------------
Query ID: red20
Query: {'username': 'RED,', 'age': '20'}
Data: [{'tweetid': 123, 'username': 'user1'}, {'tweetid': 456, 'username': 'user2'}]
Timestamp: 2024-04-21 19:38:26
--------------------
Query ID: blue21
Query: {'username': 'BLUE,', 'age': '21'}
Data: [{'tweetid': 456, 'username': 'user3'}, {'tweetid': 789, 'username': 'user4'}]
Timestamp: 2024-04-21 19:38:27
--------------------


In [72]:
cache.export_to_json("cache_data1.json")

In [73]:
# Retrieve data from cache
cached_data = cache.get_from_cache(query)
if cached_data:
    print("Data found in cache:")
    print(json.dumps(cached_data, indent=4))
else:
    print("Data not found in cache.")

Data found in cache:
{
    "ID": "red20",
    "Query": {
        "username": "RED,",
        "age": "20"
    },
    "data": [
        {
            "tweetid": 123,
            "username": "user1"
        },
        {
            "tweetid": 456,
            "username": "user2"
        }
    ],
    "timestamp": 1713742706.3014503
}
Refreshing cache...
Cache refreshed at: 2024-04-21 19:48:07
Refreshing cache...
Cache refreshed at: 2024-04-21 19:52:29
Refreshing cache...
Cache refreshed at: 2024-04-21 19:53:33
Refreshing cache...
Cache refreshed at: 2024-04-21 19:53:51
Refreshing cache...
Cache refreshed at: 2024-04-21 19:54:15
Refreshing cache...
Cache refreshed at: 2024-04-21 19:59:02
Refreshing cache...
Cache refreshed at: 2024-04-21 20:01:20
Refreshing cache...
Cache refreshed at: 2024-04-21 20:04:52
Refreshing cache...
Cache refreshed at: 2024-04-21 20:07:27
Refreshing cache...
Cache refreshed at: 2024-04-21 20:08:22


Exception in thread Thread-10 (_auto_refresh_cache):
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\ProgramData\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\adish\AppData\Local\Temp\ipykernel_15972\2244249987.py", line 73, in _auto_refresh_cache
  File "C:\Users\adish\AppData\Local\Temp\ipykernel_15972\2244249987.py", line 21, in _refresh_cache
KeyError: '{"age": "20", "username": "red"}'


Refreshing cache...


Exception in thread Thread-11 (_auto_refresh_cache):
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\ProgramData\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\adish\AppData\Local\Temp\ipykernel_15972\1613476361.py", line 73, in _auto_refresh_cache
  File "C:\Users\adish\AppData\Local\Temp\ipykernel_15972\1613476361.py", line 21, in _refresh_cache
KeyError: '{"age": "20", "username": "red"}'


Refreshing cache...


...

Refreshing cache...


Exception in thread Thread-8 (_auto_refresh_cache):
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\ProgramData\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\adish\AppData\Local\Temp\ipykernel_15972\2912041832.py", line 73, in _auto_refresh_cache
  File "C:\Users\adish\AppData\Local\Temp\ipykernel_15972\2912041832.py", line 21, in _refresh_cache
KeyError: '{"age": "20", "username": "red"}'


Refreshing cache...


Exception in thread Thread-9 (_auto_refresh_cache):
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\ProgramData\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\adish\AppData\Local\Temp\ipykernel_15972\78655480.py", line 73, in _auto_refresh_cache
  File "C:\Users\adish\AppData\Local\Temp\ipykernel_15972\78655480.py", line 21, in _refresh_cache
KeyError: '{"age": "20", "username": "red"}'


Refreshing cache...
Refreshing cache...
Cache refreshed at: 2024-04-22 00:46:43
Refreshing cache...
Cache refreshed at: 2024-04-22 00:47:38
